# Pruebas de software y aseguramiento de la calidad
# Actividad A6.2
## Nombre: Misael López Sánchez
## Matricula: A01796906

In [1]:
import os
import json
import sys

In [2]:
# Importamos google drive donde se almacenan los códigos y archivos
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Código

In [3]:
# 1. Ruta de trabajo
WORK_DIR = '/content/drive/MyDrive/PruebasSoftware2026/A6.2_ArchivosApoyo'

# 2. Creación del directorio
if not os.path.exists(WORK_DIR):
    os.makedirs(WORK_DIR)
    print(f"Directorio creado: {WORK_DIR}")
else:
    print(f"Directorio existente: {WORK_DIR}")

# 3. Cambiar el directorio de trabajo
os.chdir(WORK_DIR)
print(f"Directorio de trabajo actual: {os.getcwd()}")

Directorio existente: /content/drive/MyDrive/PruebasSoftware2026/A6.2_ArchivosApoyo
Directorio de trabajo actual: /content/drive/MyDrive/PruebasSoftware2026/A6.2_ArchivosApoyo


`hotel_system.py`

In [4]:
%%writefile hotel_system.py
"""
Sistema para gestionar Hoteles, Clientes y Reservaciones.
Persiste los datos en archivos JSON en un directorio específico.
"""

import os
import json

# Definir ruta base
BASE_DIR = '/content/drive/MyDrive/PruebasSoftware2026/A6.2_ArchivosApoyo'


class FileManager:
    """Clase auxiliar para manejar operaciones de archivos."""

    @staticmethod
    def get_filepath(filename):
        """Devuelve la ruta absoluta de un archivo."""
        return os.path.join(BASE_DIR, filename)

    @staticmethod
    def load_data(filename):
        """Carga los datos de un archivo JSON."""
        filepath = FileManager.get_filepath(filename)
        if not os.path.exists(filepath):
            return []
        try:
            with open(filepath, 'r', encoding='utf-8') as file:
                return json.load(file)
        except (json.JSONDecodeError, IOError):
            # Línea 31 corregida para PEP-8
            print(f"Error: No se pudo leer '{filepath}'. "
                  "Devolviendo lista vacía.")
            return []

    @staticmethod
    def save_data(filename, data):
        """Guarda los datos en un archivo JSON."""
        filepath = FileManager.get_filepath(filename)
        try:
            with open(filepath, 'w', encoding='utf-8') as file:
                json.dump(data, file, indent=4)
        except IOError:
            print(f"Error: No se pudo guardar en '{filepath}'.")


class Hotel:
    """Clase para gestionar las entidades de Hotel."""
    FILE = 'hotels.json'

    def __init__(self, hotel_id, name, location, rooms):
        self.hotel_id = hotel_id
        self.name = name
        self.location = location
        self.rooms = rooms

    def to_dict(self):
        """Devuelve la representación en diccionario del hotel."""
        return self.__dict__

    @classmethod
    def create_hotel(cls, hotel_id, name, location, rooms):
        """Crea un nuevo hotel y lo guarda en el archivo."""
        hotels = FileManager.load_data(cls.FILE)
        if any(h['hotel_id'] == hotel_id for h in hotels):
            print(f"Error: El ID del hotel {hotel_id} ya existe.")
            return
        new_hotel = cls(hotel_id, name, location, rooms)
        hotels.append(new_hotel.to_dict())
        FileManager.save_data(cls.FILE, hotels)
        print(f"Hotel '{name}' creado exitosamente.")

    @classmethod
    def delete_hotel(cls, hotel_id):
        """Elimina un hotel por su ID."""
        hotels = FileManager.load_data(cls.FILE)
        new_list = [h for h in hotels if h['hotel_id'] != hotel_id]
        if len(hotels) == len(new_list):
            print(f"Error: ID del hotel {hotel_id} no encontrado.")
        else:
            FileManager.save_data(cls.FILE, new_list)
            print(f"ID del hotel {hotel_id} eliminado.")

    @classmethod
    def display_hotel_info(cls, hotel_id):
        """Muestra la información de un hotel específico."""
        hotels = FileManager.load_data(cls.FILE)
        hotel = next((h for h in hotels if h['hotel_id'] == hotel_id), None)
        if hotel:
            print(f"Información del Hotel: {hotel}")
            return hotel
        print(f"Error: ID del hotel {hotel_id} no encontrado.")
        return None

    @classmethod
    def modify_hotel_info(cls, hotel_id, **kwargs):
        """Modifica los atributos de un hotel existente."""
        hotels = FileManager.load_data(cls.FILE)
        found = False
        for hotel in hotels:
            if hotel['hotel_id'] == hotel_id:
                hotel.update(kwargs)
                found = True
                break

        if found:
            FileManager.save_data(cls.FILE, hotels)
            print(f"ID del hotel {hotel_id} actualizado.")
        else:
            print(f"Error: ID del hotel {hotel_id} no encontrado.")

    @classmethod
    def reserve_room(cls, hotel_id):
        """Decrementa las habitaciones disponibles de un hotel."""
        hotels = FileManager.load_data(cls.FILE)
        for hotel in hotels:
            if hotel['hotel_id'] == hotel_id:
                if hotel['rooms'] > 0:
                    hotel['rooms'] -= 1
                    FileManager.save_data(cls.FILE, hotels)
                    return True
                # Línea 120 corregida para PEP-8
                print("Error: No hay habitaciones disponibles "
                      f"en el Hotel {hotel_id}.")
                return False
        print(f"Error: ID del hotel {hotel_id} no encontrado.")
        return False

    @classmethod
    def cancel_reservation(cls, hotel_id):
        """Incrementa las habitaciones disponibles de un hotel."""
        hotels = FileManager.load_data(cls.FILE)
        for hotel in hotels:
            if hotel['hotel_id'] == hotel_id:
                hotel['rooms'] += 1
                FileManager.save_data(cls.FILE, hotels)
                return
        print(f"Error: ID del hotel {hotel_id} no encontrado.")


class Customer:
    """Clase para gestionar las entidades de Cliente."""
    FILE = 'customers.json'

    def __init__(self, customer_id, name, email):
        self.customer_id = customer_id
        self.name = name
        self.email = email

    def to_dict(self):
        """Devuelve la representación en diccionario del cliente."""
        return self.__dict__

    @classmethod
    def create_customer(cls, customer_id, name, email):
        """Crea un nuevo cliente."""
        customers = FileManager.load_data(cls.FILE)
        if any(c['customer_id'] == customer_id for c in customers):
            print(f"Error: El ID del cliente {customer_id} ya existe.")
            return
        new_cust = cls(customer_id, name, email)
        customers.append(new_cust.to_dict())
        FileManager.save_data(cls.FILE, customers)
        print(f"Cliente '{name}' creado exitosamente.")

    @classmethod
    def delete_customer(cls, customer_id):
        """Elimina un cliente."""
        customers = FileManager.load_data(cls.FILE)
        new_list = [c for c in customers if c['customer_id'] != customer_id]
        if len(customers) == len(new_list):
            print(f"Error: ID del cliente {customer_id} no encontrado.")
        else:
            FileManager.save_data(cls.FILE, new_list)
            print(f"ID del cliente {customer_id} eliminado.")

    @classmethod
    def display_customer_info(cls, customer_id):
        """Muestra la información del cliente."""
        customers = FileManager.load_data(cls.FILE)
        cust = next(
            (c for c in customers if c['customer_id'] == customer_id),
            None
        )
        if cust:
            print(f"Información del Cliente: {cust}")
            return cust
        print(f"Error: ID del cliente {customer_id} no encontrado.")
        return None

    @classmethod
    def modify_customer_info(cls, customer_id, **kwargs):
        """Modifica la información del cliente."""
        customers = FileManager.load_data(cls.FILE)
        found = False
        for cust in customers:
            if cust['customer_id'] == customer_id:
                cust.update(kwargs)
                found = True
                break

        if found:
            FileManager.save_data(cls.FILE, customers)
            print(f"ID del cliente {customer_id} actualizado.")
        else:
            print(f"Error: ID del cliente {customer_id} no encontrado.")


class Reservation:
    """Clase para gestionar las Reservaciones."""
    FILE = 'reservations.json'

    def __init__(self, reservation_id, customer_id, hotel_id):
        self.reservation_id = reservation_id
        self.customer_id = customer_id
        self.hotel_id = hotel_id

    def to_dict(self):
        """Devuelve la representación en diccionario de la reservación."""
        return self.__dict__

    @classmethod
    def create_reservation(cls, reservation_id, customer_id, hotel_id):
        """Crea una reservación si el hotel y el cliente existen."""
        # Verificar que el cliente existe
        cust = Customer.display_customer_info(customer_id)
        if not cust:
            print("Fallo en la reservación: Cliente no encontrado.")
            return

        # Verificar que el hotel existe y reservar habitación
        if not Hotel.reserve_room(hotel_id):
            # Línea 229 corregida para PEP-8
            print("Fallo en la reservación: Hotel no encontrado "
                  "o sin habitaciones.")
            return

        reservations = FileManager.load_data(cls.FILE)
        new_res = cls(reservation_id, customer_id, hotel_id)
        reservations.append(new_res.to_dict())
        FileManager.save_data(cls.FILE, reservations)
        print(f"Reservación {reservation_id} creada exitosamente.")

    @classmethod
    def cancel_reservation(cls, reservation_id):
        """Cancela una reservación y libera la habitación."""
        reservations = FileManager.load_data(cls.FILE)
        res = next(
            (r for r in reservations if r['reservation_id'] == reservation_id),
            None
        )
        if not res:
            print(f"Error: Reservación {reservation_id} no encontrada.")
            return

        # Liberar la habitación en el hotel
        Hotel.cancel_reservation(res['hotel_id'])

        # Eliminar reservación
        new_list = [
            r for r in reservations
            if r['reservation_id'] != reservation_id
        ]
        FileManager.save_data(cls.FILE, new_list)
        print(f"Reservación {reservation_id} cancelada.")

Overwriting hotel_system.py


# Generación de los datos de insumo para el programa

In [5]:
# Ruta base
BASE_DIR = '/content/drive/MyDrive/PruebasSoftware2026/A6.2_ArchivosApoyo'

# Datos iniciales
hotels_data = [
    {"hotel_id": "H1", "name": "Hotel California", "location": "USA", "rooms": 10},
    {"hotel_id": "H2", "name": "Grand Hotel", "location": "Spain", "rooms": 5}
]

customers_data = [
    {"customer_id": "C1", "name": "John Doe", "email": "john@example.com"},
    {"customer_id": "C2", "name": "Jane Smith", "email": "jane@example.com"}
]

# Función auxiliar para escribir en la ruta correcta
def write_json(filename, data):
    filepath = os.path.join(BASE_DIR, filename)
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4)
    print(f"Archivo generado: {filepath}")

# Generación de archivos
write_json('hotels.json', hotels_data)
write_json('customers.json', customers_data)
write_json('reservations.json', [])

Archivo generado: /content/drive/MyDrive/PruebasSoftware2026/A6.2_ArchivosApoyo/hotels.json
Archivo generado: /content/drive/MyDrive/PruebasSoftware2026/A6.2_ArchivosApoyo/customers.json
Archivo generado: /content/drive/MyDrive/PruebasSoftware2026/A6.2_ArchivosApoyo/reservations.json


Verificación de la creación de los archivos correctos.

In [6]:
files = os.listdir('/content/drive/MyDrive/PruebasSoftware2026/A6.2_ArchivosApoyo')
print("Archivos en la carpeta:", files)

Archivos en la carpeta: ['__pycache__', 'hotel_system.py', 'hotels.json', 'customers.json', 'reservations.json', 'test_hotel_system.py', '.coverage']


# Ponemos a prueba el código generado.

In [7]:
WORK_DIR = '/content/drive/MyDrive/PruebasSoftware2026/A6.2_ArchivosApoyo'
if WORK_DIR not in sys.path:
    sys.path.append(WORK_DIR)
os.chdir(WORK_DIR)

# 1. Importar tus clases desde el archivo 'hotel_system.py'
try:
    from hotel_system import Hotel, Customer, Reservation
    print("Módulo 'hotel_system' importado correctamente.\n")
except ImportError as e:
    print(f"Error al importar: {e}")


# --- BLOQUE DE PRUEBAS de métodos PUT/GET ---

print("--- 1. PRUEBA DE ESCRITURA ---")
# Crear un Hotel Nuevo
print(">> Registrando 'Hotel Pruebas'...")
Hotel.create_hotel("H-TEST-01", "Hotel Python", "Cancún", 2)

# Crear un Cliente Nuevo
print(">> Registrando Cliente 'Usuario Test'...")
Customer.create_customer("C-TEST-01", "Usuario Test", "test@itesm.mx")

# Crear una Reservación (Esto debería restar 1 habitación al hotel)
print(">> Generando Reservación...")
Reservation.create_reservation("R-TEST-01", "C-TEST-01", "H-TEST-01")


print("\n--- 2. PRUEBA DE LECTURA Y ESTADO ---")
# Verificar info del Hotel
hotel_info = Hotel.display_hotel_info("H-TEST-01")

# Verificar info del Cliente
cust_info = Customer.display_customer_info("C-TEST-01")


print("\n--- 3. VERIFICACIÓN DE PERSISTENCIA (JSON) ---")
# Leeremos directamente el archivo JSON para asegurar que se guardó en disco
def leer_json_raw(nombre_archivo):
    with open(nombre_archivo, 'r') as f:
        data = json.load(f)
        # Filtramos solo los datos de prueba para mostrar
        items = [d for d in data if "TEST" in str(d.values())]
        print(f"Contenido en {nombre_archivo} (filtrado):")
        print(json.dumps(items, indent=2))

leer_json_raw('hotels.json')
leer_json_raw('reservations.json')

print("\n--- PRUEBAS FINALIZADAS ---")

Módulo 'hotel_system' importado correctamente.

--- 1. PRUEBA DE ESCRITURA ---
>> Registrando 'Hotel Pruebas'...
Hotel 'Hotel Python' creado exitosamente.
>> Registrando Cliente 'Usuario Test'...
Cliente 'Usuario Test' creado exitosamente.
>> Generando Reservación...
Información del Cliente: {'customer_id': 'C-TEST-01', 'name': 'Usuario Test', 'email': 'test@itesm.mx'}
Reservación R-TEST-01 creada exitosamente.

--- 2. PRUEBA DE LECTURA Y ESTADO ---
Información del Hotel: {'hotel_id': 'H-TEST-01', 'name': 'Hotel Python', 'location': 'Cancún', 'rooms': 1}
Información del Cliente: {'customer_id': 'C-TEST-01', 'name': 'Usuario Test', 'email': 'test@itesm.mx'}

--- 3. VERIFICACIÓN DE PERSISTENCIA (JSON) ---
Contenido en hotels.json (filtrado):
[
  {
    "hotel_id": "H-TEST-01",
    "name": "Hotel Python",
    "location": "Canc\u00fan",
    "rooms": 1
  }
]
Contenido en reservations.json (filtrado):
[
  {
    "reservation_id": "R-TEST-01",
    "customer_id": "C-TEST-01",
    "hotel_id": "H-

# Pruebas de calidad del software

In [8]:
!pip install flake8 pylint

In [9]:
# Pruebas con Flake8
print("--- INICIANDO ANÁLISIS FLAKE8 ---")
!flake8 hotel_system.py
print("\n--- FIN DE ANÁLISIS ---")

--- INICIANDO ANÁLISIS FLAKE8 ---

--- FIN DE ANÁLISIS ---


In [10]:
# Pruebas con Pylint
print("--- INICIANDO ANÁLISIS PYLINT ---")
!pylint hotel_system.py

--- INICIANDO ANÁLISIS PYLINT ---

--------------------------------------------------------------------
Your code has been rated at 10.00/10 (previous run: 10.00/10, +0.00)



## Pruebas test negativas.

In [11]:
%%writefile test_hotel_system.py
import unittest
import os
import json
from hotel_system import Hotel, Customer, Reservation, FileManager

class TestHotelSystem(unittest.TestCase):
    """
    Suite de Pruebas Completa: Escenarios Positivos y Negativos.
    """

    def setUp(self):
        """Restablece los archivos JSON a un estado limpio antes de CADA prueba."""
        self.hotels_file = FileManager.get_filepath('hotels.json')
        self.customers_file = FileManager.get_filepath('customers.json')
        self.reservations_file = FileManager.get_filepath('reservations.json')

        with open(self.hotels_file, 'w') as f: json.dump([], f)
        with open(self.customers_file, 'w') as f: json.dump([], f)
        with open(self.reservations_file, 'w') as f: json.dump([], f)

    # ==========================================
    # CASOS POSITIVOS (Camino Feliz)
    # ==========================================

    def test_hotel_operations_positive(self):
        """Positivo: Crear, Leer, Actualizar y Eliminar un Hotel."""
        # 1. Crear
        Hotel.create_hotel("H1", "Plaza", "CDMX", 10)
        hotel = Hotel.display_hotel_info("H1")
        self.assertIsNotNone(hotel)
        self.assertEqual(hotel['name'], "Plaza")

        # 2. Actualizar
        Hotel.modify_hotel_info("H1", name="Plaza Central", rooms=15)
        hotel_updated = Hotel.display_hotel_info("H1")
        self.assertEqual(hotel_updated['name'], "Plaza Central")
        self.assertEqual(hotel_updated['rooms'], 15)

        # 3. Eliminar
        Hotel.delete_hotel("H1")
        hotel_deleted = Hotel.display_hotel_info("H1")
        self.assertIsNone(hotel_deleted)

    def test_customer_operations_positive(self):
        """Positivo: Crear, Leer, Actualizar y Eliminar un Cliente."""
        # 1. Crear
        Customer.create_customer("C1", "Carlos", "carlos@mail.com")
        cust = Customer.display_customer_info("C1")
        self.assertIsNotNone(cust)
        self.assertEqual(cust['name'], "Carlos")

        # 2. Actualizar
        Customer.modify_customer_info("C1", email="carlos_nuevo@mail.com")
        cust_updated = Customer.display_customer_info("C1")
        self.assertEqual(cust_updated['email'], "carlos_nuevo@mail.com")

        # 3. Eliminar
        Customer.delete_customer("C1")
        cust_deleted = Customer.display_customer_info("C1")
        self.assertIsNone(cust_deleted)

    def test_reservation_operations_positive(self):
        """Positivo: Crear y Cancelar una Reservación (Verifica inventario de habitaciones)."""
        # Configuración inicial
        Hotel.create_hotel("H1", "Resort", "Cancun", 5)
        Customer.create_customer("C1", "Ana", "ana@mail.com")

        # 1. Crear Reservación
        Reservation.create_reservation("R1", "C1", "H1")
        hotel_after_res = Hotel.display_hotel_info("H1")
        self.assertEqual(hotel_after_res['rooms'], 4, "Las habitaciones deben disminuir en 1")

        # 2. Cancelar Reservación
        Reservation.cancel_reservation("R1")
        hotel_after_cancel = Hotel.display_hotel_info("H1")
        self.assertEqual(hotel_after_cancel['rooms'], 5, "Las habitaciones deben aumentar en 1")

    # ==========================================
    # CASOS NEGATIVOS (Los 5 requeridos por rúbrica)
    # ==========================================

    def test_duplicate_hotel_creation(self):
        """Negativo: Intentar crear un hotel con un ID duplicado."""
        Hotel.create_hotel("H1", "Test Hotel", "Mexico", 10)
        Hotel.create_hotel("H1", "Duplicate Hotel", "USA", 20)
        hotels = FileManager.load_data('hotels.json')
        self.assertEqual(len(hotels), 1)

    def test_delete_non_existent_customer(self):
        """Negativo: Intentar eliminar un cliente que no existe."""
        Customer.create_customer("C1", "Juan", "juan@mail.com")
        Customer.delete_customer("C999")
        customers = FileManager.load_data('customers.json')
        self.assertEqual(len(customers), 1)

    def test_create_reservation_invalid_customer(self):
        """Negativo: Intentar reservar con un ID de cliente inexistente."""
        Hotel.create_hotel("H1", "Resort", "Cancun", 5)
        Reservation.create_reservation("R1", "C-GHOST", "H1")
        reservations = FileManager.load_data('reservations.json')
        self.assertEqual(len(reservations), 0)

    def test_create_reservation_invalid_hotel(self):
        """Negativo: Intentar reservar en un hotel inexistente."""
        Customer.create_customer("C1", "Ana", "ana@mail.com")
        Reservation.create_reservation("R1", "C1", "H-GHOST")
        reservations = FileManager.load_data('reservations.json')
        self.assertEqual(len(reservations), 0)

    def test_cancel_non_existent_reservation(self):
        """Negativo: Intentar cancelar una reservación que no existe."""
        Hotel.create_hotel("H1", "Resort", "Cancun", 5)
        Customer.create_customer("C1", "Ana", "ana@mail.com")
        Reservation.create_reservation("R1", "C1", "H1")
        Reservation.cancel_reservation("R-FAKE")
        reservations = FileManager.load_data('reservations.json')
        self.assertEqual(len(reservations), 1)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

Overwriting test_hotel_system.py


In [12]:
!pip install coverage

In [13]:
# 1. Corre las pruebas usando 'coverage'
!coverage run -m unittest test_hotel_system.py

# 2. Imprime el reporte en consola
print("\n--- REPORTE DE COBERTURA ---")
!coverage report -m hotel_system.py

Hotel 'Resort' creado exitosamente.
Cliente 'Ana' creado exitosamente.
Información del Cliente: {'customer_id': 'C1', 'name': 'Ana', 'email': 'ana@mail.com'}
Reservación R1 creada exitosamente.
Error: Reservación R-FAKE no encontrada.
.Hotel 'Resort' creado exitosamente.
Error: ID del cliente C-GHOST no encontrado.
Fallo en la reservación: Cliente no encontrado.
.Cliente 'Ana' creado exitosamente.
Información del Cliente: {'customer_id': 'C1', 'name': 'Ana', 'email': 'ana@mail.com'}
Error: ID del hotel H-GHOST no encontrado.
Fallo en la reservación: Hotel no encontrado o sin habitaciones.
.Cliente 'Carlos' creado exitosamente.
Información del Cliente: {'customer_id': 'C1', 'name': 'Carlos', 'email': 'carlos@mail.com'}
ID del cliente C1 actualizado.
Información del Cliente: {'customer_id': 'C1', 'name': 'Carlos', 'email': 'carlos_nuevo@mail.com'}
ID del cliente C1 eliminado.
Error: ID del cliente C1 no encontrado.
.Cliente 'Juan' creado exitosamente.
Error: ID del cliente C999 no encont